In [34]:
# LIBRERÍAS NECESARIAS
import importlib
import os

def instalar_si_no_existe(libreria):
    try:
        importlib.import_module(libreria)
        print(f"{libreria} ya está instalada.")
    except ImportError:
        print(f"Instalando {libreria}...")
        os.system(f"pip install {libreria}")

# Lista de librerías necesarias
librerias = ['pandas', 'numpy', 'requests']

for libreria in librerias:
    instalar_si_no_existe(libreria)

import pandas as pd
import numpy as np
import requests

pandas ya está instalada.
numpy ya está instalada.
requests ya está instalada.


In [35]:
# DIRECTORIOS DE TRABAJO
# URLS de los repositorios de GitHub
url_dgvg_github = "https://raw.githubusercontent.com/Margalida-MB/TFM/refs/heads/main/Datos/Datos%20en%20bruto/DGVG/"
url_ine_github = "https://raw.githubusercontent.com/Margalida-MB/TFM/refs/heads/main/Datos/Datos%20en%20bruto/INE/"

# Directorio de los datos procesados (local)
notebook_dir = os.getcwd()
ruta_datos_procesados = os.path.join(notebook_dir, 'Datos procesados')

# Crear el directorio 'Datos procesados' si no existe
if not os.path.exists(ruta_datos_procesados):
    os.makedirs(ruta_datos_procesados)

# Datos del Portal Estadístico de la Delegación del Gobierno contra la Violencia de Género

## Procesar los archivos

In [36]:
# Función para aplicar las transformaciones a cada archivo
def procesar_csv(archivo):
    # Cargar el archivo CSV en un DataFrame
    df = pd.read_csv(archivo)
    
    # Mostrar los primeros registros del DataFrame antes de la transformación
    print(f"Datos originales de {archivo}:")
    display(df.head())
    
    # Convertir a string la columna 'Año', eliminar el string 'Año ' y convertir a integer
    if 'Año' in df.columns:
        df['Año'] = df['Año'].astype(str).str.replace('Año ', '').astype(int)
    
    # Convertir Mes de texto a número (en formato texto para conservar el 0 inicial)
    if 'Mes' in df.columns:
        mapa_mes = {
            'Enero': '01', 'Febrero': '02', 'Marzo': '03', 'Abril': '04', 'Mayo': '05', 'Junio': '06',
            'Julio': '07', 'Agosto': '08', 'Septiembre': '09', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'
        }
        df['Mes'] = df['Mes'].map(mapa_mes).astype(str)
    
    # Transformar la columna Trimestre
    if 'Trimestre' in df.columns:
        mapa_trimestre = {'Primero': 'T1', 'Segundo': 'T2', 'Tercero': 'T3', 'Cuarto': 'T4'}
        df['Trimestre'] = df['Trimestre'].map(mapa_trimestre)

    # Añadir la columna 'Trimestre' si no existe y si 'Mes' está presente
    if 'Mes' in df.columns and 'Trimestre' not in df.columns:
        # Crear la columna 'Trimestre' a partir de la columna 'Mes'
        df['Trimestre'] = df['Mes'].apply(lambda x: 'T1' if x in ['01', '02', '03'] else
                                                ('T2' if x in ['04', '05', '06'] else
                                                ('T3' if x in ['07', '08', '09'] else
                                                ('T4' if x in ['10', '11', '12'] else ''))))

    # Agregar la columna 'Mes' con valores NaN en los dataframes que no la tienen
    if 'Mes' not in df.columns:
        df['Mes'] = pd.NA

    df['Mes'] = df['Mes'].astype(str)

    # Agregar la columna 'Trimestre' con valores NaN en los dataframes que no la tienen
    if 'Trimestre' not in df.columns:
        df['Trimestre'] = pd.NA

    # Agregar la columna 'Provincia' con valores NaN en los dataframes que no la tienen
    if 'Provincia' not in df.columns:
        df['Provincia'] = pd.NA

    # Convertir las columnas especificadas a tipo integer, si existen
    columnas_a_convertir = [
        "Total consultas pertinentes",
        "Llamadas pertinentes",
        "WhatsApp pertinentes",
        "Correos electrónicos pertinentes",
        "Chats pertinentes",
        "Número de ayudas concedidas Art 27",
        "Número de ayudas para cambio de residencia",
        "Número de autorizaciones concedidas",
        "Número de contratos bonificados",
        "Número de contratos de sustitución",
        "Número de denuncias por violencia de género",
        "Instalaciones acumuladas",
        "Desinstalaciones acumuladas",
        "Dispositivos activos",
        "Número de menores víctimas mortales",
        "Número de mujeres víctimas mortales",
        "Número de hijos e hijas menores huérfanos",
        "Número de perceptoras",
        "Número de casos",
        "Número de casos con protección policial",
        "Usuarias activas",
        "Altas",
        "Bajas",
        "Número de órdenes de protección"
    ]

    for columna in columnas_a_convertir:
        if columna in df.columns:
            df[columna] = pd.to_numeric(df[columna], errors='coerce').fillna(0).astype(int)
    
    # Mostrar los primeros registros del DataFrame después de la transformación
    print(f"Datos transformados de {archivo}:")
    display(df.head())
    
    # Devolver el dataframe transformado
    return df

# Lista de archivos CSV a procesar
archivos = [
    'servicio016.csv', 
    'concesiones_residencia_y_trabajo.csv',
    'ayudas_concedidas.csv',
    'ayudas_cambio_residencia.csv',
    'renta.csv',
    'contratos.csv',
    'denuncias.csv',
    'dispositivos_seguimiento.csv',
    'victimas_menores.csv',
    'victimas_mujeres.csv',
    'sistema_viogen.csv',
    'usuarias_atenpro.csv',
    'ordenes_proteccion.csv'
]

# Construir la lista completa de rutas de archivos
archivos = [os.path.join(url_dgvg_github, archivo) for archivo in archivos]

# Procesar cada archivo y almacenar los resultados en un diccionario con nombre de archivo como clave
dfs = {}
for archivo in archivos:
    # Obtener solo el nombre del archivo sin la ruta
    nombre_archivo = os.path.basename(archivo)
    # Procesar el archivo y asignarlo al diccionario con el nombre del archivo
    dfs[nombre_archivo] = procesar_csv(archivo)

Datos originales de https://raw.githubusercontent.com/Margalida-MB/TFM/refs/heads/main/Datos/Datos%20en%20bruto/DGVG/servicio016.csv:


,Año,Trimestre,Mes,Comunidad autónoma,Provincia,Total consultas pertinentes,Llamadas pertinentes,WhatsApp pertinentes,Correos electrónicos pertinentes,Chats pertinentes
0,2007,Tercero,Septiembre,Andalucía,Almería,3.0,3.0,0.0,0.0,0.0
1,2007,Tercero,Septiembre,Andalucía,Cádiz,2.0,2.0,0.0,0.0,0.0
2,2007,Tercero,Septiembre,Andalucía,Córdoba,10.0,10.0,0.0,0.0,0.0
3,2007,Tercero,Septiembre,Andalucía,Granada,8.0,8.0,0.0,0.0,0.0
4,2007,Tercero,Septiembre,Andalucía,Huelva,8.0,8.0,0.0,0.0,0.0


Datos transformados de https://raw.githubusercontent.com/Margalida-MB/TFM/refs/heads/main/Datos/Datos%20en%20bruto/DGVG/servicio016.csv:


,Año,Trimestre,Mes,Comunidad autónoma,Provincia,Total consultas pertinentes,Llamadas pertinentes,WhatsApp pertinentes,Correos electrónicos pertinentes,Chats pertinentes
0,2007,T3,09,Andalucía,Almería,3,3,0,0,0
1,2007,T3,09,Andalucía,Cádiz,2,2,0,0,0
2,2007,T3,09,Andalucía,Córdoba,10,10,0,0,0
3,2007,T3,09,Andalucía,Granada,8,8,0,0,0
4,2007,T3,09,Andalucía,Huelva,8,8,0,0,0


Datos originales de https://raw.githubusercontent.com/Margalida-MB/TFM/refs/heads/main/Datos/Datos%20en%20bruto/DGVG/concesiones_residencia_y_trabajo.csv:


...

,Año,Mes,Comunidad autónoma,Provincia,Altas,Bajas,Usuarias activas,Trimestre
0,2005,12,Andalucía,Almería,4,0,4,T4
1,2005,12,Andalucía,Cádiz,87,4,83,T4
2,2005,12,Andalucía,Córdoba,29,1,28,T4
3,2005,12,Andalucía,Granada,95,1,94,T4
4,2005,12,Andalucía,Huelva,25,0,25,T4


Datos originales de https://raw.githubusercontent.com/Margalida-MB/TFM/refs/heads/main/Datos/Datos%20en%20bruto/DGVG/ordenes_proteccion.csv:


,Año,Trimestre,Comunidad autónoma,Provincia,Número de órdenes de protección
0,Año 2009,Primero,Andalucía,Almería,82.0
1,Año 2009,Primero,Andalucía,Cádiz,312.0
2,Año 2009,Primero,Andalucía,Córdoba,125.0
3,Año 2009,Primero,Andalucía,Granada,163.0
4,Año 2009,Primero,Andalucía,Huelva,138.0


Datos transformados de https://raw.githubusercontent.com/Margalida-MB/TFM/refs/heads/main/Datos/Datos%20en%20bruto/DGVG/ordenes_proteccion.csv:


,Año,Trimestre,Comunidad autónoma,Provincia,Número de órdenes de protección,Mes
0,2009,T1,Andalucía,Almería,82,<NA>
1,2009,T1,Andalucía,Cádiz,312,<NA>
2,2009,T1,Andalucía,Córdoba,125,<NA>
3,2009,T1,Andalucía,Granada,163,<NA>
4,2009,T1,Andalucía,Huelva,138,<NA>


In [37]:
print("Claves del diccionario dfs:")
print(list(dfs.keys()))


Claves del diccionario dfs:
['servicio016.csv', 'concesiones_residencia_y_trabajo.csv', 'ayudas_concedidas.csv', 'ayudas_cambio_residencia.csv', 'renta.csv', 'contratos.csv', 'denuncias.csv', 'dispositivos_seguimiento.csv', 'victimas_menores.csv', 'victimas_mujeres.csv', 'sistema_viogen.csv', 'usuarias_atenpro.csv', 'ordenes_proteccion.csv']


## Unir los dataframes resultantes de procesar los archivos

In [38]:
# Realizar la fusión de todos los DataFrames en el diccionario dfs
df_portal_estadistico = None  # Inicializamos df_portal_estadistico como None

for nombre_df, df in dfs.items():
    # Eliminar la extensión '.csv' del nombre del DataFrame
    nombre_base = nombre_df.replace('.csv', '')
    
    # Renombrar las columnas, excluyendo las columnas de fusión comunes
    columnas_comunes = ['Año', 'Trimestre', 'Mes', 'Comunidad autónoma', 'Provincia']
    columnas_a_renombrar = [col for col in df.columns if col not in columnas_comunes]
    df = df.rename(columns={col: f"{nombre_base}_{col}" for col in columnas_a_renombrar})
    
    # Fusionar el DataFrame actual con df_portal_estadistico
    if df_portal_estadistico is None:
        df_portal_estadistico = df  # El primer DataFrame se asigna directamente
    else:
        df_portal_estadistico = pd.merge(
            df_portal_estadistico, df, 
            on=columnas_comunes, 
            how='outer'
        )

# Definir las columnas que deben ser convertidas a integer
columnas_a_convertir = [
    'Año',
    'servicio016_Total consultas pertinentes',
    'servicio016_Llamadas pertinentes', 'servicio016_WhatsApp pertinentes',
    'servicio016_Correos electrónicos pertinentes',
    'servicio016_Chats pertinentes',
    'concesiones_residencia_y_trabajo_Número de autorizaciones concedidas',
    'contratos_Número de contratos bonificados',
    'contratos_Número de contratos de sustitución',
    'denuncias_Número de denuncias por violencia de género',
    'dispositivos_seguimiento_Instalaciones acumuladas',
    'dispositivos_seguimiento_Desinstalaciones acumuladas',
    'dispositivos_seguimiento_Dispositivos activos',
    'victimas_menores_Número de menores víctimas mortales',
    'victimas_mujeres_Número de mujeres víctimas mortales',
    'victimas_mujeres_Número de hijos e hijas menores huérfanos',
    'sistema_viogen_Número de casos',
    'sistema_viogen_Número de casos con protección policial',
    'usuarias_atenpro_Altas', 'usuarias_atenpro_Bajas',
    'usuarias_atenpro_Usuarias activas',
    'ordenes_proteccion_Número de órdenes de protección'
]

# Convertir a Int64 las columnas que deben ser enteros, respetando los valores nulos
for columna in columnas_a_convertir:
    if columna in df_portal_estadistico.columns:  # Verificar si la columna existe en el DataFrame
        df_portal_estadistico[columna] = pd.to_numeric(df_portal_estadistico[columna], errors='coerce').astype('Int64')

# Simplificar el dataset
df_portal_estadistico = df_portal_estadistico.drop(columns=['servicio016_Llamadas pertinentes', 'servicio016_WhatsApp pertinentes', 'servicio016_Correos electrónicos pertinentes', 'servicio016_Chats pertinentes'])

# Ver el resultado final
df_portal_estadistico.head()

,Año,Trimestre,Mes,Comunidad autónoma,Provincia,servicio016_Total consultas pertinentes,concesiones_residencia_y_trabajo_Número de autorizaciones concedidas,ayudas_concedidas_Número de ayudas concedidas Art 27,ayudas_cambio_residencia_Número de ayudas para cambio de residencia,renta_Número de perceptoras,...,dispositivos_seguimiento_Dispositivos activos,victimas_menores_Número de menores víctimas mortales,victimas_mujeres_Número de mujeres víctimas mortales,victimas_mujeres_Número de hijos e hijas menores huérfanos,sistema_viogen_Número de casos,sistema_viogen_Número de casos con protección policial,usuarias_atenpro_Altas,usuarias_atenpro_Bajas,usuarias_atenpro_Usuarias activas,ordenes_proteccion_Número de órdenes de protección
0,2003,T1,01,Andalucía,Almería,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2003,T1,01,Andalucía,Granada,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,2003,T1,01,Andalucía,Málaga,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,2003,T1,01,Andalucía,Sevilla,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,2003,T1,01,Canarias,Santa Cruz de Tenerife,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,1,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [39]:
df_portal_estadistico.columns

Index(['Año', 'Trimestre', 'Mes', 'Comunidad autónoma', 'Provincia',
       'servicio016_Total consultas pertinentes',
       'concesiones_residencia_y_trabajo_Número de autorizaciones concedidas',
       'ayudas_concedidas_Número de ayudas concedidas Art 27',
       'ayudas_cambio_residencia_Número de ayudas para cambio de residencia',
       'renta_Número de perceptoras',
       'contratos_Número de contratos bonificados',
       'contratos_Número de contratos de sustitución',
       'denuncias_Número de denuncias por violencia de género',
       'dispositivos_seguimiento_Instalaciones acumuladas',
       'dispositivos_seguimiento_Desinstalaciones acumuladas',
       'dispositivos_seguimiento_Dispositivos activos',
       'victimas_menores_Número de menores víctimas mortales',
       'victimas_mujeres_Número de mujeres víctimas mortales',
       'victimas_mujeres_Número de hijos e hijas menores huérfanos',
       'sistema_viogen_Número de casos',
       'sistema_viogen_Número de caso

# Datos de la Estadística de Violencia de Género del Instituto Nacional de Estadística

## asuntos_incoados_infracciones.csv

In [40]:
# Leer el archivo CSV
archivo = 'asuntos_incoados_infracciones.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_asuntos_incoados_infracciones = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_asuntos_incoados_infracciones['Total'] = pd.to_numeric(
    df_asuntos_incoados_infracciones['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_asuntos_incoados_infracciones.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'asuntos_incoados_infracciones_Total Infracciones'
}, inplace=True)

# Eliminar las filas donde 'Tipo de infracción' no es 'Total Infracciones'
df_asuntos_incoados_infracciones = df_asuntos_incoados_infracciones[df_asuntos_incoados_infracciones['Tipo de infracción'] == 'Total Infracciones']

# Eliminar la columna 'Tipo de infracción'
df_asuntos_incoados_infracciones = df_asuntos_incoados_infracciones.drop(columns=['Tipo de infracción'])

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_asuntos_incoados_infracciones = df_asuntos_incoados_infracciones[df_asuntos_incoados_infracciones['Comunidad autónoma'] != 'Total Nacional']

# Convertir los valores de la columna 'Año' a un tipo entero
df_asuntos_incoados_infracciones['Año'] = pd.to_numeric(
    df_asuntos_incoados_infracciones['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Definir el diccionario de mapeo de las comunidades autónomas
mapeo_comunidades = {
    'Andalucía': 'Andalucía',
    'Canarias': 'Canarias',
    'Cataluña': 'Cataluña',
    'Comunitat Valenciana': 'Comunitat Valenciana',
    'Rioja, La': 'La Rioja',
    'País Vasco': 'País Vasco',
    'Castilla y León': 'Castilla y León',
    'Madrid, Comunidad de': 'Comunidad de Madrid',
    'Galicia': 'Galicia',
    'Castilla - La Mancha': 'Castilla-La Mancha',
    'Murcia, Región de': 'Región de Murcia',
    'Navarra, Comunidad Foral de': 'Comunidad Foral de Navarra',
    'Aragón': 'Aragón',
    'Ceuta': 'Ceuta',
    'Balears, Illes': 'Illes Balears',
    'Asturias, Principado de': 'Principado de Asturias',
    'Cantabria': 'Cantabria',
    'Extremadura': 'Extremadura',
    'Melilla': 'Melilla'
}

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_asuntos_incoados_infracciones['Comunidad autónoma'] = df_asuntos_incoados_infracciones['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_asuntos_incoados_infracciones.head()

,Comunidad autónoma,Año,asuntos_incoados_infracciones_Total Infracciones
13,Andalucía,2023,10879
14,Andalucía,2022,9364
15,Andalucía,2021,8173
16,Andalucía,2020,8172
17,Andalucía,2019,8749


## asuntos_incoados_medidas.csv

In [41]:
# Leer el archivo CSV
archivo = 'asuntos_incoados_medidas.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_asuntos_incoados_medidas = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_asuntos_incoados_medidas['Total'] = pd.to_numeric(
    df_asuntos_incoados_medidas['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_asuntos_incoados_medidas.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'asuntos_incoados_medidas_Total medidas cautelares dictadas'
}, inplace=True)

# Eliminar las filas donde 'Medidas cautelares' no es 'Total medidas cautelares dictadas'
df_asuntos_incoados_medidas = df_asuntos_incoados_medidas[df_asuntos_incoados_medidas['Medidas cautelares'] == 'Total medidas cautelares dictadas']

# Eliminar la columna 'Medidas cautelares'
df_asuntos_incoados_medidas = df_asuntos_incoados_medidas.drop(columns=['Medidas cautelares'])

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_asuntos_incoados_medidas = df_asuntos_incoados_medidas[df_asuntos_incoados_medidas['Comunidad autónoma'] != 'Total Nacional']

# Convertir los valores de la columna 'Año' a un tipo entero
df_asuntos_incoados_medidas['Año'] = pd.to_numeric(
    df_asuntos_incoados_medidas['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_asuntos_incoados_medidas['Comunidad autónoma'] = df_asuntos_incoados_medidas['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_asuntos_incoados_medidas.head()

,Comunidad autónoma,Año,asuntos_incoados_medidas_Total medidas cautelares dictadas
13,Andalucía,2023,24988
14,Andalucía,2022,21563
15,Andalucía,2021,20293
16,Andalucía,2020,21038
17,Andalucía,2019,21435


## asuntos_incoados_mujeres.csv

In [42]:
# Leer el archivo CSV
archivo = 'asuntos_incoados_mujeres.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_asuntos_incoados_mujeres = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_asuntos_incoados_mujeres['Total'] = pd.to_numeric(
    df_asuntos_incoados_mujeres['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_asuntos_incoados_mujeres.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'asuntos_incoados_Mujeres con medidas de protección'
}, inplace=True)

# Convertir los valores de la columna 'Año' a un tipo entero
df_asuntos_incoados_mujeres['Año'] = pd.to_numeric(
    df_asuntos_incoados_mujeres['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_asuntos_incoados_mujeres = df_asuntos_incoados_mujeres[df_asuntos_incoados_mujeres['Comunidad autónoma'] != 'Total Nacional']

# Eliminar las filas donde 'Grupo de edad' no es 'Todas las edades'
df_asuntos_incoados_mujeres = df_asuntos_incoados_mujeres[df_asuntos_incoados_mujeres['Grupo de edad'] == 'Todas las edades']

# Eliminar la columna 'Grupo de edad'
df_asuntos_incoados_mujeres = df_asuntos_incoados_mujeres.drop(columns=['Grupo de edad'])

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_asuntos_incoados_mujeres['Comunidad autónoma'] = df_asuntos_incoados_mujeres['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_asuntos_incoados_mujeres.head()

,Comunidad autónoma,Año,asuntos_incoados_Mujeres con medidas de protección
78,Andalucía,2023,8407
79,Andalucía,2022,7417
80,Andalucía,2021,6720
81,Andalucía,2020,6813
82,Andalucía,2019,7101


## asuntos_incoados_menores.csv

In [43]:
# Leer el archivo CSV
archivo = 'asuntos_incoados_menores.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_asuntos_incoados_menores = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_asuntos_incoados_menores['Total'] = pd.to_numeric(
    df_asuntos_incoados_menores['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_asuntos_incoados_menores.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'asuntos_incoados_Menores con medidas de protección'
}, inplace=True)

# Convertir los valores de la columna 'Año' a un tipo entero
df_asuntos_incoados_menores['Año'] = pd.to_numeric(
    df_asuntos_incoados_menores['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_asuntos_incoados_menores = df_asuntos_incoados_menores[df_asuntos_incoados_menores['Comunidad autónoma'] != 'Total Nacional']

# Eliminar las filas donde 'Sexo' no es 'Total'
df_asuntos_incoados_menores = df_asuntos_incoados_menores[df_asuntos_incoados_menores['Sexo'] == 'Total']

# Eliminar la columna 'Sexo'
df_asuntos_incoados_menores = df_asuntos_incoados_menores.drop(columns=['Sexo'])

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_asuntos_incoados_menores['Comunidad autónoma'] = df_asuntos_incoados_menores['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_asuntos_incoados_menores.head()

,Comunidad autónoma,Año,asuntos_incoados_Menores con medidas de protección
2,Andalucía,2023,352
3,Andalucía,2022,226
4,Aragón,2023,58
5,Aragón,2022,49
6,Principado de Asturias,2023,68


## asuntos_incoados_denunciados.csv

In [44]:
# Leer el archivo CSV
archivo = 'asuntos_incoados_denunciados.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_asuntos_incoados_denunciados = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_asuntos_incoados_denunciados['Total'] = pd.to_numeric(
    df_asuntos_incoados_denunciados['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_asuntos_incoados_denunciados.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'asuntos_incoados_Hombres denunciados con medidas de protección aplicadas'
}, inplace=True)

# Convertir los valores de la columna 'Año' a un tipo entero
df_asuntos_incoados_denunciados['Año'] = pd.to_numeric(
    df_asuntos_incoados_denunciados['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_asuntos_incoados_denunciados = df_asuntos_incoados_denunciados[df_asuntos_incoados_denunciados['Comunidad autónoma'] != 'Total Nacional']

# Eliminar las filas donde 'Grupo de edad' no es 'Todas las edades'
df_asuntos_incoados_denunciados = df_asuntos_incoados_denunciados[df_asuntos_incoados_denunciados['Grupo de edad'] == 'Todas las edades']

# Eliminar la columna 'Grupo de edad'
df_asuntos_incoados_denunciados = df_asuntos_incoados_denunciados.drop(columns=['Grupo de edad'])

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_asuntos_incoados_denunciados['Comunidad autónoma'] = df_asuntos_incoados_denunciados['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_asuntos_incoados_denunciados.head()

,Comunidad autónoma,Año,asuntos_incoados_Hombres denunciados con medidas de protección aplicadas
78,Andalucía,2023,8366
79,Andalucía,2022,7590
80,Andalucía,2021,6691
81,Andalucía,2020,6790
82,Andalucía,2019,7098


## sentencias_firmes_infracciones.csv

In [45]:
# Leer el archivo CSV
archivo = 'sentencias_firmes_infracciones.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_sentencias_firmes_infracciones = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_sentencias_firmes_infracciones['Total'] = pd.to_numeric(
    df_sentencias_firmes_infracciones['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_sentencias_firmes_infracciones.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'sentencias_firmes_infracciones_Total Infracciones'
}, inplace=True)

# Eliminar las filas donde 'Tipo de infracción' no es 'Total Infracciones'
df_sentencias_firmes_infracciones = df_sentencias_firmes_infracciones[df_sentencias_firmes_infracciones['Tipo de infracción'] == 'Total Infracciones']

# Eliminar la columna 'Tipo de infracción'
df_sentencias_firmes_infracciones = df_sentencias_firmes_infracciones.drop(columns=['Tipo de infracción'])

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_sentencias_firmes_infracciones = df_sentencias_firmes_infracciones[df_sentencias_firmes_infracciones['Comunidad autónoma'] != 'Total Nacional']

# Convertir los valores de la columna 'Año' a un tipo entero
df_sentencias_firmes_infracciones['Año'] = pd.to_numeric(
    df_sentencias_firmes_infracciones['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_sentencias_firmes_infracciones['Comunidad autónoma'] = df_sentencias_firmes_infracciones['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_sentencias_firmes_infracciones.head()


,Comunidad autónoma,Año,sentencias_firmes_infracciones_Total Infracciones
9,Andalucía,2023,10301
10,Andalucía,2022,9957
11,Andalucía,2021,9264
12,Andalucía,2020,7107
13,Andalucía,2019,8579


## sentencias_firmes_penas.csv

In [46]:
# Leer el archivo CSV
archivo = 'sentencias_firmes_penas.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_sentencias_firmes_penas = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_sentencias_firmes_penas['Total'] = pd.to_numeric(
    df_sentencias_firmes_penas['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_sentencias_firmes_penas.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'sentencias_firmes_penas_y_medidas_Total penas y medidas'
}, inplace=True)

# Eliminar las filas donde 'Penas y medidas' no es 'Total penas y medidas'
df_sentencias_firmes_penas = df_sentencias_firmes_penas[df_sentencias_firmes_penas['Penas y medidas'] == 'Total penas y medidas']

# Eliminar la columna 'Penas y medidas'
df_sentencias_firmes_penas = df_sentencias_firmes_penas.drop(columns=['Penas y medidas'])

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_sentencias_firmes_penas = df_sentencias_firmes_penas[df_sentencias_firmes_penas['Comunidad autónoma'] != 'Total Nacional']

# Convertir los valores de la columna 'Año' a un tipo entero
df_sentencias_firmes_penas['Año'] = pd.to_numeric(
    df_sentencias_firmes_penas['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_sentencias_firmes_penas['Comunidad autónoma'] = df_sentencias_firmes_penas['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_sentencias_firmes_penas.head()

,Comunidad autónoma,Año,sentencias_firmes_penas_y_medidas_Total penas y medidas
9,Andalucía,2023,39533
10,Andalucía,2022,38079
11,Andalucía,2021,37132
12,Andalucía,2020,28257
13,Andalucía,2019,34238


## sentencias_firmes_absueltos.csv

In [47]:
# Leer el archivo CSV
archivo = 'sentencias_firmes_absueltos.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_sentencias_firmes_absueltos = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_sentencias_firmes_absueltos['Total'] = pd.to_numeric(
    df_sentencias_firmes_absueltos['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_sentencias_firmes_absueltos.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'sentencias_firmes_Total absueltos'
}, inplace=True)

# Convertir los valores de la columna 'Año' a un tipo entero
df_sentencias_firmes_absueltos['Año'] = pd.to_numeric(
    df_sentencias_firmes_absueltos['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_sentencias_firmes_absueltos = df_sentencias_firmes_absueltos[df_sentencias_firmes_absueltos['Comunidad autónoma'] != 'Total Nacional']

# Eliminar las filas donde 'Grupo de edad' no es 'Todas las edades'
df_sentencias_firmes_absueltos = df_sentencias_firmes_absueltos[df_sentencias_firmes_absueltos['Grupo de edad '] == 'Todas las edades']

# Eliminar la columna 'Grupo de edad'
df_sentencias_firmes_absueltos = df_sentencias_firmes_absueltos.drop(columns=['Grupo de edad '])

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_sentencias_firmes_absueltos['Comunidad autónoma'] = df_sentencias_firmes_absueltos['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_sentencias_firmes_absueltos.head()

,Comunidad autónoma,Año,sentencias_firmes_Total absueltos
54,Andalucía,2023,<NA>
55,Andalucía,2022,<NA>
56,Andalucía,2021,969
57,Andalucía,2020,777
58,Andalucía,2019,1237


## sentencias_firmes_condenados.csv

In [48]:
# Leer el archivo CSV
archivo = 'sentencias_firmes_condenados.csv'
ruta_archivo = os.path.join(url_ine_github, archivo)
df_sentencias_firmes_condenados = pd.read_csv(ruta_archivo, sep=';', thousands='.', decimal=',')

# Convertir 'Total' a un tipo entero que soporte nulos
df_sentencias_firmes_condenados['Total'] = pd.to_numeric(
    df_sentencias_firmes_condenados['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_sentencias_firmes_condenados.rename(columns={
    'Comunidades y ciudades autónomas': 'Comunidad autónoma',
    'Periodo': 'Año',
    'Total': 'sentencias_firmes_Total condenados'
}, inplace=True)

# Convertir los valores de la columna 'Año' a un tipo entero
df_sentencias_firmes_condenados['Año'] = pd.to_numeric(
    df_sentencias_firmes_condenados['Año'], errors='coerce'
).astype('Int64')  # También aseguramos que 'Año' sea 'Int64'

# Eliminar las filas donde 'Comunidad autónoma' es 'Total Nacional'
df_sentencias_firmes_condenados = df_sentencias_firmes_condenados[df_sentencias_firmes_condenados['Comunidad autónoma'] != 'Total Nacional']

# Eliminar las filas donde 'Grupo de edad' no es 'Todas las edades'
df_sentencias_firmes_condenados = df_sentencias_firmes_condenados[df_sentencias_firmes_condenados['Grupo de edad'] == 'Todas las edades']

# Eliminar la columna 'Grupo de edad'
df_sentencias_firmes_condenados = df_sentencias_firmes_condenados.drop(columns=['Grupo de edad'])

# Aplicar el mapeo a la columna 'Comunidades autónomas'
df_sentencias_firmes_condenados['Comunidad autónoma'] = df_sentencias_firmes_condenados['Comunidad autónoma'].map(mapeo_comunidades)

# Visualizar el DataFrame transformado
df_sentencias_firmes_condenados.head()

,Comunidad autónoma,Año,sentencias_firmes_Total condenados
54,Andalucía,2023,6933
55,Andalucía,2022,6856
56,Andalucía,2021,6457
57,Andalucía,2020,5113
58,Andalucía,2019,6110


## Combinar dataframes de la Estadística de Violencia de Género del Instituto Nacional de Estadística

In [49]:
# Lista de DataFrames a fusionar
dataframes = [
    df_sentencias_firmes_condenados,
    df_sentencias_firmes_absueltos,
    df_sentencias_firmes_penas,
    df_sentencias_firmes_infracciones,
    df_asuntos_incoados_medidas,
    df_asuntos_incoados_infracciones,
    df_asuntos_incoados_denunciados,
    df_asuntos_incoados_menores,
    df_asuntos_incoados_mujeres
]

# Realizar el merge progresivo
df_ine = dataframes[0]  # Tomamos el primer DataFrame como base
for df in dataframes[1:]:
    df_ine = pd.merge(df_ine, df, on=['Comunidad autónoma', 'Año'], how='outer')

# Agregar las columnas con valores NaN (utilizando pd.NA)
df_ine['Mes'] = pd.NA
df_ine['Trimestre'] = pd.NA
df_ine['Provincia'] = pd.NA

# Mostrar el df resultante
df_ine.head()

,Comunidad autónoma,Año,sentencias_firmes_Total condenados,sentencias_firmes_Total absueltos,sentencias_firmes_penas_y_medidas_Total penas y medidas,sentencias_firmes_infracciones_Total Infracciones,asuntos_incoados_medidas_Total medidas cautelares dictadas,asuntos_incoados_infracciones_Total Infracciones,asuntos_incoados_Hombres denunciados con medidas de protección aplicadas,asuntos_incoados_Menores con medidas de protección,asuntos_incoados_Mujeres con medidas de protección,Mes,Trimestre,Provincia
0,Andalucía,2011,<NA>,<NA>,<NA>,<NA>,23392,9328,7724,<NA>,7780,<NA>,<NA>,<NA>
1,Andalucía,2012,<NA>,<NA>,<NA>,<NA>,21733,8357,7111,<NA>,7161,<NA>,<NA>,<NA>
2,Andalucía,2013,<NA>,<NA>,<NA>,<NA>,19862,7863,6761,<NA>,6800,<NA>,<NA>,<NA>
3,Andalucía,2014,<NA>,<NA>,<NA>,<NA>,20564,8137,6983,<NA>,7012,<NA>,<NA>,<NA>
4,Andalucía,2015,4773,1567,26088,6492,20550,8290,6984,<NA>,7014,<NA>,<NA>,<NA>


In [50]:
df_ine.columns

Index(['Comunidad autónoma', 'Año', 'sentencias_firmes_Total condenados',
       'sentencias_firmes_Total absueltos',
       'sentencias_firmes_penas_y_medidas_Total penas y medidas',
       'sentencias_firmes_infracciones_Total Infracciones',
       'asuntos_incoados_medidas_Total medidas cautelares dictadas',
       'asuntos_incoados_infracciones_Total Infracciones',
       'asuntos_incoados_Hombres denunciados con medidas de protección aplicadas',
       'asuntos_incoados_Menores con medidas de protección',
       'asuntos_incoados_Mujeres con medidas de protección', 'Mes',
       'Trimestre', 'Provincia'],
      dtype='object')

In [51]:
df_ine.columns

Index(['Comunidad autónoma', 'Año', 'sentencias_firmes_Total condenados',
       'sentencias_firmes_Total absueltos',
       'sentencias_firmes_penas_y_medidas_Total penas y medidas',
       'sentencias_firmes_infracciones_Total Infracciones',
       'asuntos_incoados_medidas_Total medidas cautelares dictadas',
       'asuntos_incoados_infracciones_Total Infracciones',
       'asuntos_incoados_Hombres denunciados con medidas de protección aplicadas',
       'asuntos_incoados_Menores con medidas de protección',
       'asuntos_incoados_Mujeres con medidas de protección', 'Mes',
       'Trimestre', 'Provincia'],
      dtype='object')

# Generar el conjunto de datos integrado

In [52]:
# Realizar la fusión de los dos DataFrames usando las columnas comunes
df_final = pd.merge(df_portal_estadistico, df_ine, 
                    on=['Año', 'Trimestre', 'Mes', 'Comunidad autónoma', 'Provincia'], 
                    how='outer') 

# # Crear la columna 'Fecha' como string, asegurando que si 'Año' o 'Mes' son <NA>, se deje como nulo
# df_final['Fecha'] = df_final.apply(
#     lambda row: f"{str(row['Año'])}-{str(row['Mes']).zfill(2)}-01" 
#     if pd.notna(row['Año']) and pd.notna(row['Mes']) and row['Mes'] != "<NA>" else None, 
#     axis=1
# )

# # Crear la columna 'Año-Trimestre' como string, asegurando que si 'Año' o 'Trimestre' son <NA>, se deje como nulo
# df_final['Año-Trimestre'] = df_final.apply(
#     lambda row: f"{str(row['Año'])} {str(row['Trimestre'])}" if pd.notna(row['Año']) and pd.notna(row['Trimestre']) else None, 
#     axis=1
# )

# Reemplazar el string '<NA>' por NaN en la columna 'Mes'
df_final['Mes'] = df_final['Mes'].replace('<NA>', np.nan)

# Crear la columna 'Año-Trimestre-Mes' como string, asegurando que si 'Trimestre' o 'Mes' son <NA>, se muestren literalmente como NA
df_final['Año-Trimestre-Mes'] = df_final.apply(
    lambda row: f"{str(row['Año']) if pd.notna(row['Año']) else 'NA'}-" +
                f"{'NA' if pd.isna(row['Trimestre']) or str(row['Trimestre']).strip() in ['', '<NA>'] else str(row['Trimestre'])}-" +
                f"{str(row['Mes']) if pd.notna(row['Mes']) and row['Mes'] != '<NA>' else 'NA'}",
    axis=1
)

# Verificar el resultado
df_final.head()

,Año,Trimestre,Mes,Comunidad autónoma,Provincia,servicio016_Total consultas pertinentes,concesiones_residencia_y_trabajo_Número de autorizaciones concedidas,ayudas_concedidas_Número de ayudas concedidas Art 27,ayudas_cambio_residencia_Número de ayudas para cambio de residencia,renta_Número de perceptoras,...,sentencias_firmes_Total condenados,sentencias_firmes_Total absueltos,sentencias_firmes_penas_y_medidas_Total penas y medidas,sentencias_firmes_infracciones_Total Infracciones,asuntos_incoados_medidas_Total medidas cautelares dictadas,asuntos_incoados_infracciones_Total Infracciones,asuntos_incoados_Hombres denunciados con medidas de protección aplicadas,asuntos_incoados_Menores con medidas de protección,asuntos_incoados_Mujeres con medidas de protección,Año-Trimestre-Mes
0,2003,T1,01,Andalucía,Almería,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2003-T1-01
1,2003,T1,01,Andalucía,Granada,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2003-T1-01
2,2003,T1,01,Andalucía,Málaga,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2003-T1-01
3,2003,T1,01,Andalucía,Sevilla,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2003-T1-01
4,2003,T1,01,Canarias,Santa Cruz de Tenerife,<NA>,<NA>,NaN,NaN,NaN,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2003-T1-01


In [53]:
# Exportar el df a csv
df_final.to_csv(f"{ruta_datos_procesados}/datos_procesados_simplificado.csv", index=False)

# Imprimir mensaje de confirmación
print(f"El archivo 'datos_procesados_simplificado.csv' se ha guardado en {ruta_datos_procesados}.")

El archivo 'datos_procesados_simplificado.csv' se ha guardado en d:\Marga\OneDrive - UNIR\PER9191 VIDAMA\TFE - Trabajo Fin de Estudios\Datos\Procesado de datos\Datos procesados.


In [54]:
df_final.columns

Index(['Año', 'Trimestre', 'Mes', 'Comunidad autónoma', 'Provincia',
       'servicio016_Total consultas pertinentes',
       'concesiones_residencia_y_trabajo_Número de autorizaciones concedidas',
       'ayudas_concedidas_Número de ayudas concedidas Art 27',
       'ayudas_cambio_residencia_Número de ayudas para cambio de residencia',
       'renta_Número de perceptoras',
       'contratos_Número de contratos bonificados',
       'contratos_Número de contratos de sustitución',
       'denuncias_Número de denuncias por violencia de género',
       'dispositivos_seguimiento_Instalaciones acumuladas',
       'dispositivos_seguimiento_Desinstalaciones acumuladas',
       'dispositivos_seguimiento_Dispositivos activos',
       'victimas_menores_Número de menores víctimas mortales',
       'victimas_mujeres_Número de mujeres víctimas mortales',
       'victimas_mujeres_Número de hijos e hijas menores huérfanos',
       'sistema_viogen_Número de casos',
       'sistema_viogen_Número de caso

# Datos del la Estadística del Continua de Población del Instituto Nacional de Estadística

## poblacion.csv

In [55]:
# Crear una lista de nombres de archivos
archivos = [f'poblacion_parte{i}.csv' for i in range(1, 7)]

# Leer y concatenar los archivos CSV
df_poblacion = pd.concat(
    [pd.read_csv(os.path.join(url_ine_github, archivo), sep=';', thousands='.', decimal=',') for archivo in archivos],
    ignore_index=True
)

# Convertir 'Total' a un tipo entero que soporte nulos
df_poblacion['Total'] = pd.to_numeric(
    df_poblacion['Total'], errors='coerce'
).astype('Int64')  # 'Int64' soporta NaN como valores nulos

# Cambiar el título de las columnas
df_poblacion.rename(columns={
    'Generación': 'Edad',
    'Provincias': 'Provincia'
}, inplace=True)

# Eliminar las filas donde 'Provincia' es 'Total Nacional'
df_poblacion = df_poblacion[df_poblacion['Provincia'] != 'Total Nacional']

# Eliminar las filas donde 'Edad' es 'Todas las edades'
df_poblacion = df_poblacion[df_poblacion['Edad'] != 'Todas las edades']

# Eliminar los strings ' y más años' y ' años', y convertir la columna 'Edad' a entero
df_poblacion['Edad'] = df_poblacion['Edad'].str.replace(' y más años', '', regex=False)
df_poblacion['Edad'] = df_poblacion['Edad'].str.replace(' años', '', regex=False)
df_poblacion['Edad'] = df_poblacion['Edad'].str.replace(' año', '', regex=False)
df_poblacion['Edad'] = df_poblacion['Edad'].astype(int)

# Eliminar las filas donde 'Periodo' contiene '1 de octubre', '1 de julio' o '1 de abril'
df_poblacion = df_poblacion[~df_poblacion['Periodo'].str.contains('1 de octubre|1 de julio|1 de abril', case=False, na=False)]

# Eliminar los 3 primeros caracteres de la columna 'Provincia'
df_poblacion['Provincia'] = df_poblacion['Provincia'].str.slice(start=3)

# Extraer el año (últimos 4 caracteres)
df_poblacion['Año'] = df_poblacion['Periodo'].str[-4:].astype(int)

# Eliminar las filas donde 'Año' es '2002'
df_poblacion = df_poblacion[df_poblacion['Año'] != 2002]

# Eliminar las filas donde 'Provincia' es 'Total Nacional'
df_poblacion = df_poblacion[df_poblacion['Provincia'] != 'Total Nacional']

# Eliminar la columna 'Periodo'
df_poblacion = df_poblacion.drop(columns=['Periodo'])

# Transformar a formato wide
df_poblacion = df_poblacion.pivot_table(
    index=['Año', 'Provincia', 'Edad'],      # Índices
    columns= ['Sexo'],                       # Columnas a expandir
    values='Total'                           # Valores a utilizar
).reset_index()

# Convertir a valor entero
df_poblacion['Total'] = df_poblacion['Total'].astype(int)
df_poblacion['Hombres'] = df_poblacion['Hombres'].astype(int)
df_poblacion['Mujeres'] = df_poblacion['Mujeres'].astype(int)

# Diccionario de mapeo
mapa_provincias = {
    'Palmas, Las': 'Las Palmas',
    'Valencia/València': 'Valencia/València',
    'Balears, Illes': 'Illes Balears',
    'Coruña, A': 'A Coruña',
    'Rioja, La': 'La Rioja',    
}

# Modificar la columna 'Provincia' usando el mapeo
df_poblacion['Provincia'] = df_poblacion['Provincia'].replace(mapa_provincias)

# Diccionario de mapeo de provincias a comunidades autónomas
mapeo_comunidad = {
    'Almería': 'Andalucía', 'Granada': 'Andalucía', 'Málaga': 'Andalucía', 'Sevilla': 'Andalucía',
    'Santa Cruz de Tenerife': 'Canarias', 'Barcelona': 'Cataluña', 'Valencia/València': 'Comunitat Valenciana',
    'La Rioja': 'La Rioja', 'Bizkaia': 'País Vasco', 'Las Palmas': 'Canarias', 'Palencia': 'Castilla y León',
    'Lleida': 'Cataluña', 'Tarragona': 'Cataluña', 'Madrid': 'Comunidad de Madrid', 'A Coruña': 'Galicia',
    'Cádiz': 'Andalucía', 'Jaén': 'Andalucía', 'Burgos': 'Castilla y León', 'Segovia': 'Castilla y León',
    'Soria': 'Castilla y León', 'Zamora': 'Castilla y León', 'Ciudad Real': 'Castilla-La Mancha',
    'Alicante/Alacant': 'Comunitat Valenciana', 'Lugo': 'Galicia', 'Murcia': 'Región de Murcia', 'León': 'Castilla y León',
    'Navarra': 'Comunidad Foral de Navarra', 'Teruel': 'Aragón', 'Valladolid': 'Castilla y León',
    'Albacete': 'Castilla-La Mancha', 'Toledo': 'Castilla-La Mancha', 'Girona': 'Cataluña', 'Pontevedra': 'Galicia',
    'Zaragoza': 'Aragón', 'Salamanca': 'Castilla y León', 'Ceuta': 'Ceuta', 'Illes Balears': 'Illes Balears',
    'Asturias': 'Principado de Asturias', 'Cantabria': 'Cantabria', 'Castellón/Castelló': 'Comunitat Valenciana',
    'Ávila': 'Castilla y León', 'Córdoba': 'Andalucía', 'Badajoz': 'Extremadura', 'Gipuzkoa': 'País Vasco',
    'Cáceres': 'Extremadura', 'Ourense': 'Galicia', 'Guadalajara': 'Castilla-La Mancha', 'Huesca': 'Aragón',
    'Huelva': 'Andalucía', 'Araba/Álava': 'País Vasco', 'Cuenca': 'Castilla-La Mancha', 'Melilla': 'Melilla'
}

# Añadir la columna 'Comunidad autónoma' usando el mapeo
df_poblacion['Comunidad autónoma'] = df_poblacion['Provincia'].map(mapeo_comunidad)

# Mostrar el df
df_poblacion.head()

Sexo,Año,Provincia,Edad,Hombres,Mujeres,Total,Comunidad autónoma
0,2003,Albacete,0,1861,1844,3705,Castilla-La Mancha
1,2003,Albacete,1,1886,1895,3781,Castilla-La Mancha
2,2003,Albacete,2,1893,1863,3757,Castilla-La Mancha
3,2003,Albacete,3,1923,1784,3708,Castilla-La Mancha
4,2003,Albacete,4,1948,1811,3760,Castilla-La Mancha


In [56]:
# Exportar el df a csv
df_poblacion.to_csv(f"{ruta_datos_procesados}/poblacion_procesado.csv", index=False)

# Imprimir mensaje de confirmación
print(f"El archivo 'poblacion_procesado.csv' se ha guardado en {ruta_datos_procesados}.")

El archivo 'poblacion_procesado.csv' se ha guardado en d:\Marga\OneDrive - UNIR\PER9191 VIDAMA\TFE - Trabajo Fin de Estudios\Datos\Procesado de datos\Datos procesados.
